In [1]:
import pickle
import pandas as pd
import numpy as np
import xgboost as xgb
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load data
data = []
with open('../data/HCD35_pos.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [3]:
# divide target and features
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [4]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [5]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [6]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

In [7]:
# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_HCD35_pos.h5')

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 2s 2ms/step - loss: 1.9807 - acc: 0.2716 - val_loss: 1.4502 - val_acc: 0.2798
Epoch 2/100
1340/1340 [==============================] - 1s 628us/step - loss: 1.8038 - acc: 0.2701 - val_loss: 1.4709 - val_acc: 0.3333
Epoch 3/100
1340/1340 [==============================] - 1s 625us/step - loss: 1.7190 - acc: 0.2985 - val_loss: 1.4186 - val_acc: 0.3423
Epoch 4/100
1340/1340 [==============================] - 1s 620us/step - loss: 1.7501 - acc: 0.2739 - val_loss: 1.3578 - val_acc: 0.3571
Epoch 5/100
1340/1340 [==============================] - 1s 620us/step - loss: 1.6938 - acc: 0.3112 - val_loss: 1.3087 - val_acc: 0.3839
Epoch 6/100
1340/1340 [==============================] - 1s 619us/step - loss: 1.5389 - acc: 0.3366 - val_loss: 1.2864 - val_acc: 0.3899
Epoch 7/100
1340/1340 [==============================] - 1s 617us/step - loss: 1.5191 - acc: 0.3642 - val_loss: 1.2539 - val_acc: 0.3

1340/1340 [==============================] - 1s 615us/step - loss: 0.2633 - acc: 0.9127 - val_loss: 1.0634 - val_acc: 0.6726
Epoch 61/100
1340/1340 [==============================] - 1s 610us/step - loss: 0.2486 - acc: 0.9187 - val_loss: 1.0736 - val_acc: 0.6756
Epoch 62/100
1340/1340 [==============================] - 1s 624us/step - loss: 0.2455 - acc: 0.9000 - val_loss: 1.0585 - val_acc: 0.6786
Epoch 63/100
1340/1340 [==============================] - 1s 651us/step - loss: 0.2125 - acc: 0.9254 - val_loss: 1.0452 - val_acc: 0.6994
Epoch 64/100
1340/1340 [==============================] - 1s 667us/step - loss: 0.2200 - acc: 0.9254 - val_loss: 1.0386 - val_acc: 0.7113
Epoch 65/100
1340/1340 [==============================] - 1s 639us/step - loss: 0.2044 - acc: 0.9366 - val_loss: 1.0334 - val_acc: 0.7113
Epoch 66/100
1340/1340 [==============================] - 1s 626us/step - loss: 0.1827 - acc: 0.9328 - val_loss: 1.0318 - val_acc: 0.7054
Epoch 67/100
1340/1340 [=======================

In [8]:
#predict
score = model.evaluate(X_test, y_test_for_keras, verbose=0)

j = []
elapsed_time = time.time() - start
j.append(elapsed_time)

f = []
f.append(score[1]*100)

In [9]:
score[1]

0.7232142857142857

In [10]:
# load data
data = []
with open('../data/HCD45_pos.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [11]:
# divide target and features
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [12]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [13]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [14]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_HCD45_pos.h5')

#predict
score = model.evaluate(X_test, y_test_for_keras, verbose=0)

elapsed_time = time.time() - start
j.append(elapsed_time)

f.append(score[1]*100)

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 3s 2ms/step - loss: 1.9326 - acc: 0.2552 - val_loss: 1.4605 - val_acc: 0.2827
Epoch 2/100
1340/1340 [==============================] - 1s 709us/step - loss: 1.8309 - acc: 0.2701 - val_loss: 1.4606 - val_acc: 0.3244
Epoch 3/100
1340/1340 [==============================] - 1s 726us/step - loss: 1.7561 - acc: 0.2784 - val_loss: 1.4392 - val_acc: 0.3482
Epoch 4/100
1340/1340 [==============================] - 1s 720us/step - loss: 1.6477 - acc: 0.2813 - val_loss: 1.3723 - val_acc: 0.3601
Epoch 5/100
1340/1340 [==============================] - 1s 709us/step - loss: 1.6721 - acc: 0.2776 - val_loss: 1.3129 - val_acc: 0.3780
Epoch 6/100
1340/1340 [==============================] - 1s 741us/step - loss: 1.5234 - acc: 0.3299 - val_loss: 1.2584 - val_acc: 0.3601
Epoch 7/100
1340/1340 [==============================] - 1s 760us/step - loss: 1.4856 - acc: 0.3590 - val_loss: 1.2315 - val_acc: 0.3

1340/1340 [==============================] - 1s 750us/step - loss: 0.2471 - acc: 0.9119 - val_loss: 0.8844 - val_acc: 0.6994
Epoch 61/100
1340/1340 [==============================] - 1s 714us/step - loss: 0.2731 - acc: 0.9119 - val_loss: 0.9001 - val_acc: 0.6577
Epoch 62/100
1340/1340 [==============================] - 1s 716us/step - loss: 0.2521 - acc: 0.9134 - val_loss: 0.8962 - val_acc: 0.6518
Epoch 63/100
1340/1340 [==============================] - 1s 715us/step - loss: 0.2430 - acc: 0.9119 - val_loss: 0.8860 - val_acc: 0.6696
Epoch 64/100
1340/1340 [==============================] - 1s 716us/step - loss: 0.2467 - acc: 0.9224 - val_loss: 0.8806 - val_acc: 0.6935
Epoch 65/100
1340/1340 [==============================] - 1s 732us/step - loss: 0.2526 - acc: 0.9172 - val_loss: 0.8790 - val_acc: 0.6994
Epoch 66/100
1340/1340 [==============================] - 1s 702us/step - loss: 0.2225 - acc: 0.9269 - val_loss: 0.8805 - val_acc: 0.7113
Epoch 67/100
1340/1340 [=======================

In [15]:
# load data
data = []
with open('../data/HCD65_pos.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [16]:
# divide target and features
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [17]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [18]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [19]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_HCD65_pos.h5')

#predict
score = model.evaluate(X_test, y_test_for_keras, verbose=0)

elapsed_time = time.time() - start
j.append(elapsed_time)

f.append(score[1]*100)

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 3s 2ms/step - loss: 1.9340 - acc: 0.2612 - val_loss: 1.4266 - val_acc: 0.2946
Epoch 2/100
1340/1340 [==============================] - 1s 720us/step - loss: 1.8289 - acc: 0.2575 - val_loss: 1.3672 - val_acc: 0.3304
Epoch 3/100
1340/1340 [==============================] - 1s 718us/step - loss: 1.7796 - acc: 0.2978 - val_loss: 1.3388 - val_acc: 0.3333
Epoch 4/100
1340/1340 [==============================] - 1s 714us/step - loss: 1.7010 - acc: 0.2687 - val_loss: 1.2982 - val_acc: 0.3542
Epoch 5/100
1340/1340 [==============================] - 1s 726us/step - loss: 1.6005 - acc: 0.3134 - val_loss: 1.2477 - val_acc: 0.3780
Epoch 6/100
1340/1340 [==============================] - 1s 700us/step - loss: 1.5114 - acc: 0.3187 - val_loss: 1.2200 - val_acc: 0.3988
Epoch 7/100
1340/1340 [==============================] - 1s 702us/step - loss: 1.5471 - acc: 0.3224 - val_loss: 1.2085 - val_acc: 0.4

1340/1340 [==============================] - 1s 725us/step - loss: 0.4865 - acc: 0.7866 - val_loss: 0.9129 - val_acc: 0.6280
Epoch 61/100
1340/1340 [==============================] - 1s 741us/step - loss: 0.4791 - acc: 0.8179 - val_loss: 0.9021 - val_acc: 0.6310
Epoch 62/100
1340/1340 [==============================] - 1s 807us/step - loss: 0.4785 - acc: 0.8060 - val_loss: 0.8988 - val_acc: 0.6310
Epoch 63/100
1340/1340 [==============================] - 1s 791us/step - loss: 0.4361 - acc: 0.8306 - val_loss: 0.9026 - val_acc: 0.6250
Epoch 64/100
1340/1340 [==============================] - 1s 722us/step - loss: 0.4300 - acc: 0.8388 - val_loss: 0.9106 - val_acc: 0.6071
Epoch 65/100
1340/1340 [==============================] - 1s 705us/step - loss: 0.4245 - acc: 0.8276 - val_loss: 0.9156 - val_acc: 0.6071
Epoch 66/100
1340/1340 [==============================] - 1s 729us/step - loss: 0.4659 - acc: 0.8239 - val_loss: 0.9180 - val_acc: 0.5923
Epoch 67/100
1340/1340 [=======================

In [20]:
# load data
data = []
with open('../data/HCD35_neg.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [21]:
# divide target and features
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [22]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [23]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [24]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_HCD35_neg.h5')

#predict
score = model.evaluate(X_test, y_test_for_keras, verbose=0)

elapsed_time = time.time() - start
j.append(elapsed_time)

f.append(score[1]*100)

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 2s 5ms/step - loss: 2.0594 - acc: 0.3324 - val_loss: 1.2999 - val_acc: 0.4362
Epoch 2/100
373/373 [==============================] - 0s 204us/step - loss: 1.8601 - acc: 0.2976 - val_loss: 1.2854 - val_acc: 0.4362
Epoch 3/100
373/373 [==============================] - 0s 243us/step - loss: 1.8735 - acc: 0.2842 - val_loss: 1.3202 - val_acc: 0.4574
Epoch 4/100
373/373 [==============================] - 0s 214us/step - loss: 1.9252 - acc: 0.3003 - val_loss: 1.3558 - val_acc: 0.4574
Epoch 5/100
373/373 [==============================] - 0s 218us/step - loss: 1.6347 - acc: 0.3566 - val_loss: 1.3598 - val_acc: 0.4681
Epoch 6/100
373/373 [==============================] - 0s 211us/step - loss: 1.6729 - acc: 0.3753 - val_loss: 1.3463 - val_acc: 0.4894
Epoch 7/100
373/373 [==============================] - 0s 207us/step - loss: 1.7338 - acc: 0.3592 - val_loss: 1.3003 - val_acc: 0.5106
Epoch 8/100


Epoch 61/100
373/373 [==============================] - 0s 230us/step - loss: 0.6577 - acc: 0.7534 - val_loss: 0.9166 - val_acc: 0.6596
Epoch 62/100
373/373 [==============================] - 0s 220us/step - loss: 0.6735 - acc: 0.7239 - val_loss: 0.9088 - val_acc: 0.6596
Epoch 63/100
373/373 [==============================] - 0s 207us/step - loss: 0.5967 - acc: 0.7426 - val_loss: 0.9057 - val_acc: 0.6596
Epoch 64/100
373/373 [==============================] - 0s 211us/step - loss: 0.6025 - acc: 0.7373 - val_loss: 0.9103 - val_acc: 0.6596
Epoch 65/100
373/373 [==============================] - 0s 227us/step - loss: 0.5916 - acc: 0.7587 - val_loss: 0.9154 - val_acc: 0.6489
Epoch 66/100
373/373 [==============================] - 0s 220us/step - loss: 0.6246 - acc: 0.7507 - val_loss: 0.9190 - val_acc: 0.6489
Epoch 67/100
373/373 [==============================] - 0s 221us/step - loss: 0.5594 - acc: 0.7802 - val_loss: 0.9223 - val_acc: 0.6489
Epoch 68/100
373/373 [==========================

In [25]:
# load data
data = []
with open('../data/HCD45_neg.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [26]:
# divide target and features
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [27]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [28]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [29]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_HCD45_neg.h5')

#predict
score = model.evaluate(X_test, y_test_for_keras, verbose=0)

elapsed_time = time.time() - start
j.append(elapsed_time)

f.append(score[1]*100)

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 2s 6ms/step - loss: 2.0211 - acc: 0.2815 - val_loss: 1.3210 - val_acc: 0.4255
Epoch 2/100
373/373 [==============================] - 0s 310us/step - loss: 1.8936 - acc: 0.3190 - val_loss: 1.2780 - val_acc: 0.4468
Epoch 3/100
373/373 [==============================] - 0s 283us/step - loss: 1.8786 - acc: 0.2949 - val_loss: 1.3458 - val_acc: 0.4362
Epoch 4/100
373/373 [==============================] - 0s 285us/step - loss: 1.7519 - acc: 0.3646 - val_loss: 1.3964 - val_acc: 0.4574
Epoch 5/100
373/373 [==============================] - 0s 312us/step - loss: 1.6608 - acc: 0.3753 - val_loss: 1.4154 - val_acc: 0.4574
Epoch 6/100
373/373 [==============================] - 0s 301us/step - loss: 1.5765 - acc: 0.3941 - val_loss: 1.3900 - val_acc: 0.4574
Epoch 7/100
373/373 [==============================] - 0s 290us/step - loss: 1.6477 - acc: 0.3592 - val_loss: 1.3546 - val_acc: 0.4574
Epoch 8/100


Epoch 61/100
373/373 [==============================] - 0s 300us/step - loss: 0.6705 - acc: 0.7051 - val_loss: 1.0051 - val_acc: 0.6596
Epoch 62/100
373/373 [==============================] - 0s 292us/step - loss: 0.6531 - acc: 0.7265 - val_loss: 1.0123 - val_acc: 0.6596
Epoch 63/100
373/373 [==============================] - 0s 275us/step - loss: 0.6296 - acc: 0.7373 - val_loss: 1.0195 - val_acc: 0.6702


In [30]:
# load data
data = []
with open('../data/HCD65_neg.pickle', mode='rb') as fp:
    df = pickle.load(fp)

In [31]:
# divide target and features
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [32]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [33]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [34]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_HCD65_neg.h5')

#predict
score = model.evaluate(X_test, y_test_for_keras, verbose=0)

elapsed_time = time.time() - start
j.append(elapsed_time)

f.append(score[1]*100)

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 2s 6ms/step - loss: 2.0835 - acc: 0.2895 - val_loss: 1.3423 - val_acc: 0.4149
Epoch 2/100
373/373 [==============================] - 0s 392us/step - loss: 1.9243 - acc: 0.2869 - val_loss: 1.2595 - val_acc: 0.4362
Epoch 3/100
373/373 [==============================] - 0s 408us/step - loss: 1.9408 - acc: 0.2842 - val_loss: 1.2734 - val_acc: 0.5000
Epoch 4/100
373/373 [==============================] - 0s 386us/step - loss: 1.8170 - acc: 0.3271 - val_loss: 1.3196 - val_acc: 0.5106
Epoch 5/100
373/373 [==============================] - 0s 405us/step - loss: 1.7376 - acc: 0.3458 - val_loss: 1.3318 - val_acc: 0.5000
Epoch 6/100
373/373 [==============================] - 0s 389us/step - loss: 1.6870 - acc: 0.3432 - val_loss: 1.3131 - val_acc: 0.5000
Epoch 7/100
373/373 [==============================] - 0s 398us/step - loss: 1.6855 - acc: 0.3592 - val_loss: 1.2804 - val_acc: 0.5106
Epoch 8/100


Epoch 61/100
373/373 [==============================] - 0s 404us/step - loss: 0.6835 - acc: 0.7239 - val_loss: 0.8811 - val_acc: 0.6809
Epoch 62/100
373/373 [==============================] - 0s 391us/step - loss: 0.7822 - acc: 0.6702 - val_loss: 0.8850 - val_acc: 0.7128
Epoch 63/100
373/373 [==============================] - 0s 403us/step - loss: 0.6572 - acc: 0.6971 - val_loss: 0.8898 - val_acc: 0.7128
Epoch 64/100
373/373 [==============================] - 0s 398us/step - loss: 0.7126 - acc: 0.6917 - val_loss: 0.8960 - val_acc: 0.7234
Epoch 65/100
373/373 [==============================] - 0s 404us/step - loss: 0.7050 - acc: 0.6944 - val_loss: 0.9014 - val_acc: 0.7234
Epoch 66/100
373/373 [==============================] - 0s 414us/step - loss: 0.6848 - acc: 0.7078 - val_loss: 0.9075 - val_acc: 0.7234
Epoch 67/100
373/373 [==============================] - 0s 422us/step - loss: 0.6508 - acc: 0.7480 - val_loss: 0.9140 - val_acc: 0.7128
Epoch 68/100
373/373 [==========================

In [35]:
pd.DataFrame(
    [f, j], 
    index=['Accuracy', 'Time'],
    columns = [
        'HCD35_Positive','HCD45_Positive','HCD65_Positive',
        'HCD35_Negative','HCD45_Negative','HCD65_Negative'
    ]
).T.to_csv('../result/Keras.csv')